# 替换同音字

@author 何俊傑

In [21]:
from tqdm import tqdm
from pypinyin import lazy_pinyin #pypinyin
import jieba #分词用
import random #随机抽取用

## 一、读取词库

In [22]:
def read_from_file(f):
    """从文件f中读取内容，并返回每一行对应的拼音和对应词，假设文件的编码是utf-8编码"""
    content = ''
    with open(f, 'r', encoding='utf-8') as f:
        content = f.readlines()    
    homophones = [line.split('\t') for line in content] #先把所有词读进来
    
    for line in homophones: #把换行符去掉的循环
        if line[-1][-1]=='\n':
            line[-1]=line[-1][:-1]
        
    homophones = {line[0]:line[1:] for line in homophones}        
    return homophones

展示词库: 我使用了字典的形式储存词库内容。key<词语的拼音>:value<同音词语列表>

In [23]:
homophones_file_path = 'chinese_homophone_char.txt'
homophones_test = read_from_file(homophones_file_path)
homophones_test

{'de': ['的', '地', '得', '德', '嘚', '徳', '锝', '脦', '悳', '淂', '鍀', '惪', '恴', '棏'],
 'le': ['了',
  '乐',
  '叻',
  '肋',
  '樂',
  '楽',
  '仂',
  '泐',
  '簕',
  '饹',
  '竻',
  '砳',
  '玏',
  '鳓',
  '扐',
  '艻',
  '忇',
  '氻',
  '阞',
  '韷',
  '鰳',
  '餎'],
 'shi': ['是',
  '时',
  '使',
  '事',
  '式',
  '市',
  '师',
  '诗',
  '似',
  '十',
  '石',
  '屎',
  '史',
  '室',
  '试',
  '失',
  '饰',
  '视',
  '食',
  '湿',
  '识',
  '实',
  '氏',
  '世',
  '施',
  '势',
  '時',
  '始',
  '士',
  '适',
  '示',
  '狮',
  '拾',
  '尸',
  '誓',
  '释',
  '仕',
  '實',
  '侍',
  '逝',
  '驶',
  '匙',
  '拭',
  '恃',
  '噬',
  '嗜',
  '莳',
  '矢',
  '師',
  '蚀',
  '谥',
  '柿',
  '湜',
  '弑',
  '詩',
  '襹',
  '適',
  '試',
  '視',
  '識',
  '勢',
  '昰',
  '媞',
  '虱',
  '豕',
  '轼',
  '溡',
  '亊',
  '獅',
  '鍦',
  '舐',
  '濕',
  '釋',
  '実',
  '飾',
  '豉',
  '屍',
  '螫',
  '筮',
  '寔',
  '礻',
  '駛',
  '辻',
  '兘',
  '蓍',
  '铈',
  '丗',
  '埘',
  '奭',
  '蝕',
  '饣',
  '諟',
  '弒',
  '蒔',
  '溼',
  '卋',
  '煶',
  '贳',
  '浉',
  '炻',
  '鲥',
  '鉽',
  '忕',
  '遈',
  '溮',
  '笶',
  '徥',
  '鲺

## 二、开发输入词语便输出同音词的功能

使用pypinyin模块，解析句子中所有拼音

In [24]:
sentence = lazy_pinyin('小方乐高老师机械设计制造及其自动化专业孩子们的大脑大多处于未开发阶段，那么如何最大程度度激发孩子们的潜力就成为了我们所要努力追求的方向')
print(sentence)

['xiao', 'fang', 'le', 'gao', 'lao', 'shi', 'ji', 'xie', 'she', 'ji', 'zhi', 'zao', 'ji', 'qi', 'zi', 'dong', 'hua', 'zhuan', 'ye', 'hai', 'zi', 'men', 'de', 'da', 'nao', 'da', 'duo', 'chu', 'yu', 'wei', 'kai', 'fa', 'jie', 'duan', '，', 'na', 'me', 'ru', 'he', 'zui', 'da', 'cheng', 'du', 'du', 'ji', 'fa', 'hai', 'zi', 'men', 'de', 'qian', 'li', 'jiu', 'cheng', 'wei', 'le', 'wo', 'men', 'suo', 'yao', 'nu', 'li', 'zhui', 'qiu', 'de', 'fang', 'xiang']


## 三、开发替换功能

### 2.字级别的音近替换

step1. 开发具体函数 get_homophones_charactors，使得可以输入字后输出同音字列表

In [25]:
###输入字，便可输出同音字集
def get_homophones_charactors(charactor, homophones):
    """输入一个单字，可以输出其同音字列表。若词库中没有此词，便输出'no match word'"""
    
    # 取得该字的拼音
    pinyin = lazy_pinyin(charactor)[0]
    msg = ''
    try:
        charactors = homophones[pinyin]
        if charactor in charactors:
            charactors.remove(charactor)
            #只输出前十个同音字，减少词库压力
            charactors = charactors[:10] 
    except KeyError:
        #如果词库中没有此词语，便输出'no match word'
        msg = 'no match charactor'
    
    # 如果报错信息不为空，就返回报错信息
    if msg!='':
        return msg
    #如果报错信息为空，就返回同音字集合
    else:
        return charactors   

In [26]:
homophones = read_from_file('chinese_homophone_char.txt')
get_homophones_charactors('了', homophones)

['乐', '叻', '肋', '樂', '楽', '仂', '泐', '簕', '饹', '竻']

step2. 开发具体函数charactor_homophones_replace，在输入句子后输出替换同音字后的结果

In [27]:
def charactor_homophones_replace(sentence, homophones=read_from_file('chinese_homophone_char.txt')):
    """输入一个句子和同音字库后，输出一个替换了原句同音字的句子集合"""
    sentence_tobechanged = list(sentence)
    for i in range(100):
        random_index = random.choice(range(len(sentence_tobechanged)))
        charactor = sentence[random_index]
        charactor_homophones = get_homophones_charactors(charactor, homophones) #获取同音字集合
        if charactor_homophones == 'no match charactor': #如果同音字库没有这个字，就跳过此字
            continue
        else:
            random_homophone = random.choice(charactor_homophones) #随机选取同音词典中的一个同音词进行替换
            sentence_tobechanged[random_index] = random_homophone
            sentence_changed = "".join(sentence_tobechanged)
            return sentence_changed

In [28]:
sentence = '我国古代著名画家顾恺之曾曾经说过的“迁想妙得”，形象地说明了艺术从生活中汲取营养的作用'
charactor_homophones_replace(sentence)

'我国古代著名画家顾恺之曾曾经说过的“迁想妙得”，形象地说明了艺术从生活中汲取影养的作用'

## 处理文本

In [29]:
def read_json_txt(path):
    result=[]
    import json
    with open(path, 'r',encoding='utf-8') as f:
        lines = f.readlines()
    for line in tqdm(lines):
        try:
            result.append(json.loads(line))
        except:
            continue
        
    return result

In [30]:
#字音输入错误 charactor_homophones_replace(sentence)
#输入数据集所在的路径
path = './同音错别字.txt'
result = read_json_txt(path)

#换词之后的结果放在这个变量里
result_charactor_homophones_replace = []

#对于
for sentence in tqdm(result):
    try:
        change = {'通顺':charactor_homophones_replace(sentence['text']),'不通顺':charactor_homophones_replace(sentence['text'])}
        result_charactor_homophones_replace.append(change)
    except IndexError:
        continue
    
import json
with open('./同音错别字(处理完).txt', 'w', encoding='UTF-8') as f:
    for i in tqdm(result_charactor_homophones_replace):
        f.write(json.dumps(i, ensure_ascii=False)+"\n")

100%|██████████| 400633/400633 [00:02<00:00, 173350.17it/s]
